In [16]:
import pandas as pd
import numpy as np
import random
import os
import mysql.connector
from datetime import  datetime, timedelta
import time

In [3]:
DB_CONFIG = {
    "host": "localhost",  # Change if using a remote DB
    "user": "root",
    "password": "1234567890",
    "database": "test"  # Choose your schema
}

In [4]:
locations_filename = 'locations.csv'

In [ ]:
def insert_location_data():
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # Create table if not exists
        create_table_query = """
        CREATE TABLE IF NOT EXISTS locations (
            id VARCHAR(10) PRIMARY KEY,
            name VARCHAR(255),
            is_active BOOLEAN,
            selling_type TEXT
        );
        """
        cursor.execute(create_table_query)
        print("Table checked/created successfully.")

        # Insert data into the table
        for _, row in locations.iterrows():
            insert_query = """
            INSERT INTO locations (id, name, is_active, selling_type)
            VALUES (%s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE name = VALUES(name), is_active = VALUES(is_active), selling_type = VALUES(selling_type);
            """
            cursor.execute(insert_query, (row["id"], row["name"], row["is_active"], ",".join(row["selling_type"])))

        # Commit and close connection
        conn.commit()
        cursor.close()
        conn.close()
        print("Data inserted into MySQL successfully.")

    except mysql.connector.Error as err:
        print(f"Error: {err}")

In [36]:
def insert_user_data():
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # Create table if not exists
        create_table_query = """
        CREATE TABLE IF NOT EXISTS users (
                id SERIAL PRIMARY KEY,
                name VARCHAR(255) NOT NULL,
                email VARCHAR(255) UNIQUE NOT NULL,
                phone_number VARCHAR(20) NOT NULL,
                country_code VARCHAR(10) NOT NULL,
                is_monthly BOOLEAN NOT NULL,
                monthly_locations TEXT,
                cof BOOLEAN NOT NULL
        );
        """
        cursor.execute(create_table_query)
        print("Table checked/created successfully.")

        # Insert data into the table
        for _, row in user_df.iterrows():
            insert_query = """
            INSERT INTO users (name, email, phone_number, country_code, is_monthly, monthly_locations, cof)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE name = VALUES(name),
            email = VALUES(email),
            phone_number = VALUES(phone_number),
            country_code = VALUES(country_code),
            is_monthly = VALUES(is_monthly),
            monthly_locations = VALUES(monthly_locations),
            cof = VALUES(cof);
            """
            cursor.execute(insert_query, (row["name"], row["email"], row["phone_number"], row["country_code"],row["is_monthly"],",".join(row["monthly_locations"]), row["cof"]))

        # Commit and close connection
        conn.commit()
        cursor.close()
        conn.close()
        print("Data inserted into MySQL successfully.")

    except mysql.connector.Error as err:
        print(f"Error: {err}")

In [14]:
if not os.path.exists('locatons.csv'):

    data = {
        "id": ["99999", "60287", "76817", "47261","82615"],
        "name": ["Pallet", "LilyClove", "Pewter", "Lilroot", "Indigo League"],
        "is_active": [True, False, True, True, True],
        "selling_type": [["reservations"],
        ["on-demand", "reservations"],
        ["on-demand"],
        ["odv","on-demand","resevations"],
        ["odv"]]
    }

    locations = pd.DataFrame(data)

    print(locations)
    locations.to_csv(locations_filename,header=True,index=False)

      id           name  is_active                   selling_type
0  99999         Pallet       True                 [reservations]
1  60287      LilyClove      False      [on-demand, reservations]
2  76817         Pewter       True                    [on-demand]
3  47261        Lilroot       True  [odv, on-demand, resevations]
4  82615  Indigo League       True                          [odv]


In [15]:
locations = pd.read_csv(locations_filename)
locations["selling_type"] = locations["selling_type"].apply(lambda x: x.split(","))  

print(locations)

      id           name  is_active                             selling_type
0  99999         Pallet       True                       [['reservations']]
1  60287      LilyClove      False         [['on-demand',  'reservations']]
2  76817         Pewter       True                          [['on-demand']]
3  47261        Lilroot       True  [['odv',  'on-demand',  'resevations']]
4  82615  Indigo League       True                                [['odv']]


In [27]:
insert_location_data()

Table checked/created successfully.
Data inserted into MySQL successfully.


In [29]:

user_data = {
    "name": ["Alice Johnson", "Bob Smith", "Charlie Brown", "David Wilson", "Emma Davis", 
             "Fiona Martinez", "George Clark", "Hannah Lewis", "Ian Wright", "Julia Roberts"],
    "email": ["alice@example.com", "bob@example.com", "charlie@example.com", "david@example.com", "emma@example.com",
              "fiona@example.com", "george@example.com", "hannah@example.com", "ian@example.com", "julia@example.com"],
    "phone_number": ["+1234567890", "+1987654321", "+1122334455", "+1555666777", "+1444333222",
                     "+1333777888", "+1666999000", "+1222111444", "+1777888999", "+1999888777"],
    "country_code": ["US", "CA", "GB", "AU", "DE", "FR", "IN", "JP", "BR", "ZA"],
    "is_monthly": [True, False, True, True, False, True, False, True, False, True],
    "monthly_locations": [
        random.sample(["Pallet", "LilyClove", "Pewter", "Lilroot", "Indigo League"], k=random.randint(1, 3)) if is_m else []
        for is_m in [True, False, True, True, False, True, False, True, False, True]
    ],
    "cof": [random.choice([True, False]) for _ in range(10)],
}


In [31]:
user_df = pd.DataFrame(user_data)
print(user_df)

             name                email phone_number country_code  is_monthly  \
0   Alice Johnson    alice@example.com  +1234567890           US        True   
1       Bob Smith      bob@example.com  +1987654321           CA       False   
2   Charlie Brown  charlie@example.com  +1122334455           GB        True   
3    David Wilson    david@example.com  +1555666777           AU        True   
4      Emma Davis     emma@example.com  +1444333222           DE       False   
5  Fiona Martinez    fiona@example.com  +1333777888           FR        True   
6    George Clark   george@example.com  +1666999000           IN       False   
7    Hannah Lewis   hannah@example.com  +1222111444           JP        True   
8      Ian Wright      ian@example.com  +1777888999           BR       False   
9   Julia Roberts    julia@example.com  +1999888777           ZA        True   

                  monthly_locations    cof  
0                         [Lilroot]   True  
1                            

In [37]:
insert_user_data()

Table checked/created successfully.
Data inserted into MySQL successfully.


In [5]:
conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor()
methods = ['apple_pay', 'google_pay', 'hps']
data = []

for _ in range(800):
    total_amt = round(random.uniform(1, 500), 2)  # Random total amount > 0
    validated_amt = round(random.uniform(0.1, total_amt - 0.01), 2)  # Less than total_amt
    # paid_amount = round(total_amt - validated_amt, 2)  # Difference
    method = random.choice(methods)  # Random payment method
    
    data.append((total_amt, validated_amt, method))

# Insert data in batches for efficiency
query = "INSERT INTO payments (total_amt, validated_amt, method) VALUES (%s, %s, %s)"
cursor.executemany(query, data)
conn.commit()
cursor.close()
conn.close()

print("✅ Inserted 800 payments successfully!")


✅ Inserted 800 payments successfully!


In [ ]:
conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor()

# Create the orders table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS orders (
        id INT AUTO_INCREMENT PRIMARY KEY,
        order_date DATE NOT NULL,
        user_id INT NOT NULL,
        location_id VARCHAR(10) NOT NULL,
        payment_id INT UNIQUE NULL,
        FOREIGN KEY (user_id) REFERENCES users(id) ON DELETE CASCADE,
        FOREIGN KEY (location_id) REFERENCES locations(id) ON DELETE CASCADE,
        FOREIGN KEY (payment_id) REFERENCES payments(id) ON DELETE SET NULL
    )
""")
conn.commit()

# Fetch IDs
cursor.execute("SELECT id FROM users")
user_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT id FROM locations")
location_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT id FROM payments")
all_payment_ids = [row[0] for row in cursor.fetchall()]
random.shuffle(all_payment_ids)

# First 800 orders get unique payment_ids, remaining 200 get NULL
payment_ids_for_orders = all_payment_ids[:800] + [None] * 200
random.shuffle(payment_ids_for_orders)
print(payment_ids_for_orders)
# Generate 1000 order rows
orders = []
for i in range(1000):
    order_date = (datetime.today() - timedelta(days=random.randint(0, 365))).strftime('%Y-%m-%d')
    user_id = random.choice(user_ids)
    location_id = random.choice(location_ids)
    payment_id = payment_ids_for_orders[i]
    orders.append((order_date, user_id, location_id, payment_id))

insert_query = """
    INSERT INTO orders (order_date, user_id, location_id, payment_id)
    VALUES (%s, %s, %s, %s)
"""
print('here')
batch_size = 100  # Insert in chunks of 100 rows

for i in range(0, len(orders), batch_size):
    batch = orders[i:i + batch_size]
    cursor.executemany(insert_query, batch)
    conn.commit()  # Commit after each batch to release locks
    time.sleep(min = 0.1)


cursor.close()
conn.close()
print("✅ Successfully inserted 1000 orders with unique payment_ids!")

[94, 513, 757, 300, 788, 375, 745, 708, None, 417, 555, 689, 617, None, 116, 183, 616, None, 776, 152, 539, 114, None, None, 142, 261, 641, 25, None, 146, 370, 236, None, 558, 799, None, 568, 785, 42, None, 744, 110, 7, 67, 274, 156, 365, 244, 52, None, 645, 270, None, 655, 348, 769, 204, None, 294, 415, None, None, 93, 477, 245, None, 290, 250, 248, 647, 530, None, 132, 699, 284, 137, 531, 265, 725, 91, 676, 4, 53, 720, 175, 232, 624, 410, 341, 794, 786, None, 416, 611, 620, 451, None, 178, 75, None, 355, 363, 29, 134, None, 576, 60, 427, None, None, 334, None, 562, 486, 161, 543, None, 781, 454, 302, 141, 371, 197, 695, 578, 485, 719, 117, None, 165, None, 135, None, 306, 36, None, None, 319, 208, 668, 669, 450, 124, 738, 272, None, 107, 633, 43, 231, 281, 109, None, None, 44, 406, 5, 76, None, 663, None, 26, None, 654, 301, 494, None, 136, 79, 741, 634, 643, 424, 185, 401, None, 413, 28, 430, 80, 649, 212, 772, 180, 207, 694, 239, 322, None, 317, None, 507, 62, 625, 545, 253, 704, N

MySQLInterfaceError: Python type tuple cannot be converted